In [1]:
import pandas as pd

Läser in SLOSH-data för att kunna titta på frekvenstabeller.

In [2]:
fil = r'\\win.su.se\dfs\common\stressforskning.su.se\SLOSH-data\SLOSH och AMU\SLOSH\Enkätdata\SLOSH (Huvudfil, t.o.m. 2018)\Test\Slosh_all_amuar_18.xlsx'

df1 = pd.read_excel(fil)

Läser in excelfilen som jag och Martin gjort.

In [3]:
df2 = pd.read_excel('Metadata.xlsx')

Skapar en lista med de unika förekomster av variabelnamn från `df2.Namn`.

In [4]:
univ = list(df2.Namn.unique())

Skapar en lista med värden från `df2.Enkät` för varje värde i listan `univ`.

In [5]:
univenk1 = [df2.loc[df2['Namn'] == variabel, 'Enkät'].iloc[0] for variabel in univ]

Testar så att listorna har samma längd.

In [6]:
len(univ) == len(univenk1)

True

Skapar en `df3` som är en dataframe med `univ` och `univenk1` som kolumner.

In [7]:
df3 = pd.DataFrame.from_dict({'Univ': univ, 'Univenk1': univenk1})
df3.head()

,Univ,Univenk1
0,promoted_1,working
1,promoted_2,working
2,promoted_3,working
3,promoted_4,w+n
4,promoted_5,w+n


Nu skapar jag en funktion som tar en SLOSH-variabel och ser ifall den finns i working, non-working eller i båda.

In [8]:
def villkor(x):
    
    # sista symbolen i variabelnamn är vågnumret
    r = 'r_{}'.format(x[-1])
    
    # frekvenstabell för working
    w_frek = df1[df1[r] == 2][x].value_counts(dropna = False)
    
    # frekvenstabell för non-working
    nw_frek = df1[df1[r] == 1][x].value_counts(dropna = False)
    
    # w-frek finns men inte nw_frek
    if (len(w_frek) > 1) & (len(nw_frek) == 1):
        return 'working'
    
    # nw-frek finns men inte w_frek
    elif (len(w_frek) == 1) & (len(nw_frek) > 1):
        return 'non-working'
    
    # båda finns
    elif (len(w_frek) > 1) & (len(nw_frek) > 1):
        return 'w+n'
    
    # ingen finns
    else:
        return '?'

Skapar en lista till med hjälp av funktionen ovan. Denna lista skapas av information från SLOSH-datafilen och ska tjäna som ett facit för `univenk1` som är skapad från excelfilen.

In [9]:
univenk2 = [villkor(variabel) for variabel in univ]

Lägg till `univenk2` som en kolumn i `df3`.

In [10]:
df3['Univenk2'] = univenk2

Skapa en ny kolumn i `df3` och fyll den med "nej".

In [11]:
df3['Skillnad'] = 'nej'

Ändra färdet i `df3.Skillnad` till "ja" ifall informationen från excelfilen inte stämmer överens med "facit".

In [12]:
df3.loc[df3.Univenk1 != df3.Univenk2, 'Skillnad'] = 'ja'

Beskär tabellen så att enbart de fär `df3.Skillnad` = "ja" syns. Det fanns säkert 15 variabler som var felkodade men dessa har nu åtgärdats i excelfilen och när den läses in på nytt blir denna beskärning helt tom.

In [13]:
df3.loc[df3.Skillnad == 'ja', ]

,Univ,Univenk1,Univenk2,Skillnad
